Enable Kubernetes in Docker Desktop


Install Azure Functions Core Tools  https://docs.microsoft.com/en-us/azure/azure-functions/functions-run-local?tabs=windows%2Ccsharp%2Cportal%2Cbash%2Ckeda


Requirements:
- Node
- Docker Desktop with Kubernetes enabled
  

In [3]:
!npm i -g azure-functions-core-tools@3 --unsafe-perm true

npm WARN checkPermissions Missing write access to /usr/lib/node_modules
npm ERR! code EACCES
npm ERR! syscall access
npm ERR! path /usr/lib/node_modules
npm ERR! errno -13
npm ERR! Error: EACCES: permission denied, access '/usr/lib/node_modules'
npm ERR!  [Error: EACCES: permission denied, access '/usr/lib/node_modules'] {
npm ERR!   errno: -13,
npm ERR!   code: 'EACCES',
npm ERR!   syscall: 'access',
npm ERR!   path: '/usr/lib/node_modules'
npm ERR! }
npm ERR! 
npm ERR! The operation was rejected by your operating system.
npm ERR! It is likely you do not have the permissions to access this file as the current user
npm ERR! 
npm ERR! If you believe this might be a permissions issue, please double-check the
npm ERR! permissions of the file and its containing directories, or try running
npm ERR! the command again as root/Administrator.

npm ERR! A complete log of this run can be found in:
npm ERR!     /home/rondagdag/.npm/_logs/2021-09-15T21_17_23_693Z-debug.log


In [25]:
!func kubernetes install

Installing KEDA v2 in namespace 
customresourcedefinition.apiextensions.k8s.io/clustertriggerauthentications.keda.sh configured
customresourcedefinition.apiextensions.k8s.io/scaledjobs.keda.sh configured
customresourcedefinition.apiextensions.k8s.io/scaledobjects.keda.sh configured
customresourcedefinition.apiextensions.k8s.io/triggerauthentications.keda.sh configured
serviceaccount/keda-operator unchanged
clusterrole.rbac.authorization.k8s.io/keda-external-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/keda-operator configured
rolebinding.rbac.authorization.k8s.io/keda-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-hpa-controller-external-metrics unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-operator unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda:system:auth-delegator unchanged
service/keda-metrics-apiserver unchanged
deployment.apps/keda-metrics-apiserver configured
deployment.apps/keda-operator configured
apiservice

- to build folder   func init --docker

In [36]:
!rm -rf fx-edge
!func init fx-edge --docker-only --javascript

Writing Dockerfile
Writing .dockerignore


In [37]:
!func new fx-edge --template "Timer trigger" --name MyTimerTrigger --worker-runtime node 
!func new fx-edge --template "Http trigger" --name MyHttpTrigger --worker-runtime node 

Writing package.json
Writing .gitignore
Writing host.json
Writing local.settings.json
Writing /mnt/d/presentations/azfunc-percept/azfx-k8s-edge/fx-edge/.vscode/extensions.json
Select a number for template:Timer trigger
Function name: [TimerTrigger] Writing /mnt/d/presentations/azfunc-percept/azfx-k8s-edge/fx-edge/MyTimerTrigger/index.js
Writing /mnt/d/presentations/azfunc-percept/azfx-k8s-edge/fx-edge/MyTimerTrigger/readme.md
Writing /mnt/d/presentations/azfunc-percept/azfx-k8s-edge/fx-edge/MyTimerTrigger/function.json
The function "MyTimerTrigger" was created successfully from the "Timer trigger" template.
package.json already exists. Skipped!
.gitignore already exists. Skipped!
host.json already exists. Skipped!
local.settings.json already exists. Skipped!
/mnt/d/presentations/azfunc-percept/azfx-k8s-edge/fx-edge/.vscode/extensions.json already exists. Skipped!
Select a number for template:Http trigger
Function name: [HttpTrigger] Writing /mnt/d/presentations/azfunc-percept/azfx-k8s-

Change the timer to every 15 seconds
```
"schedule": "*/15 * * * * *"
```

Copy local.settings.json for azurerite connection string

change connection string to "AzureWebJobsStorage": "DefaultEndpointsProtocol=http;AccountName=devstoreaccount1;AccountKey=Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==;BlobEndpoint=http://storage-azurite-service:10000/devstoreaccount1;QueueEndpoint=http://storage-azurite-service:10001/devstoreaccount1;"


In [39]:
!cp azurite/local.settings.json fx-edge/local.settings.json
!sed -i 's="schedule": .*="schedule": "*/15 * * * * *"=g' ./fx-edge/MyTimerTrigger/function.json

Change name of docker to your dockerhub account
```
rondagdag/fxk8s:latest
```

In [40]:
!cd fx-edge && docker build --pull --rm -f "Dockerfile" -t rondagdag/fxk8s:latest "."

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 426B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 59B                                           0.0s
 => [internal] load metadata for mcr.microsoft.com/azure-functions/node:3  0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 426B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 59B                                           0.0s
 => [internal] load metadata for mcr.

In [43]:
!docker push rondagdag/fxk8s:latest

The push refers to repository [docker.io/rondagdag/fxk8s]









latest: digest: sha256:252482e468e8153c581f4bd3faeaac7cbff5e525c6d15ebc7d9cd0426671b940 size: 2219


In [44]:
!kubectl delete storage-azurite-deployment
!kubectl apply -f ./azurite/azurite.yaml

error: the server doesn't have a resource type "storage-azurite-deployment"
deployment.apps/storage-azurite-deployment unchanged
service/storage-azurite-service unchanged


In [45]:
!kubectl delete deployments fx-k8s
!cd fx-edge && func kubernetes deploy --name fx-k8s --image-name rondagdag/fxk8s:latest --dry-run > ../fx-k8s.yaml
!kubectl apply -f fx-k8s.yaml

deployment.apps "fx-k8s" deleted
secret/fx-k8s unchanged
secret/func-keys-kube-secret-fx-k8s configured
serviceaccount/fx-k8s-function-keys-identity-svc-act unchanged
role.rbac.authorization.k8s.io/functions-keys-manager-role unchanged
rolebinding.rbac.authorization.k8s.io/fx-k8s-function-keys-identity-svc-act-functions-keys-manager-rolebinding unchanged
service/fx-k8s-http unchanged
deployment.apps/fx-k8s-http unchanged
deployment.apps/fx-k8s created
scaledobject.keda.sh/fx-k8s configured


In [46]:
!kubectl get services

NAME                      TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)                         AGE
fx-k8s-http               LoadBalancer   10.107.122.116   localhost     80:32166/TCP                    20h
kubernetes                ClusterIP      10.96.0.1        <none>        443/TCP                         27h
storage-azurite-service   ClusterIP      10.96.227.211    <none>        10000/TCP,10001/TCP,10002/TCP   19h


In [47]:
!kubectl get pods

NAME                                         READY   STATUS        RESTARTS   AGE
fx-k8s-5798978758-9s648                      1/1     Terminating   1          114m
fx-k8s-5798978758-tfdww                      0/1     Running       0          20s
fx-k8s-http-6845f4dc76-fdnqv                 1/1     Running       0          114m
storage-azurite-deployment-577dbb876-9kptm   1/1     Running       0          117m


In [48]:
!kubectl get deployments

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
fx-k8s                       1/1     1            1           25s
fx-k8s-http                  1/1     1            1           115m
storage-azurite-deployment   1/1     1            1           117m


In [34]:
!kubectl logs deploy/fx-k8s --tail 20

Headers:
X-Powered-By: REDACTED
ETag: W/"a8-fxKXILNReyuR9xtKGwT8eC5QOzA"
Date: Wed, 15 Sep 2021 21:55:46 GMT
Connection: keep-alive
Content-Type: text/html; charset=utf-8
Content-Length: 168

   at Azure.Storage.Blobs.BlobRestClient.RenewLeaseAsync(String leaseId, Nullable`1 timeout, Nullable`1 ifModifiedSince, Nullable`1 ifUnmodifiedSince, String ifMatch, String ifNoneMatch, String ifTags, CancellationToken cancellationToken)
   at Azure.Storage.Blobs.Specialized.BlobLeaseClient.RenewInternal(RequestConditions conditions, Boolean async, CancellationToken cancellationToken)
   at Azure.Storage.Blobs.Specialized.BlobLeaseClient.RenewAsync(RequestConditions conditions, CancellationToken cancellationToken)
   at Microsoft.Azure.WebJobs.Script.BlobLeaseDistributedLockManager.SingletonLockHandle.RenewAsync(ILogger logger, CancellationToken cancellationToken) in /src/azure-functions-host/src/WebJobs.Script/Host/DistributedLockManagers/BlobLeaseDistributedLockManager.cs:line 334
   at Microso